In [1]:
import os
import sys

In [2]:
sys.path.append("..") # Adds higher directory to python modules path. # Works for EC2

In [12]:
import pandas as pd
import seaborn as sns
import numpy as np
import torchvision.transforms as transforms
import torch

In [26]:
from pilotnet import PilotNet
from dataset import DrivingDataset

In [6]:
data_path = '2020-10-04_06:22:49'

In [58]:
val_path = f'../data/{data_path}/val'
train_path = f'../data/{data_path}/train'

In [9]:
transform = transforms.Compose([transforms.Resize((200, 66)), transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [55]:
val_set = DrivingDataset(val_path, transform)

In [56]:
val_loader = torch.utils.data.DataLoader(val_set, shuffle=False, batch_size=32)

In [59]:
train_set = DrivingDataset(train_path, transform)

In [60]:
train_loader = torch.utils.data.DataLoader(train_set, shuffle=True, batch_size=32)

In [35]:
label_df = pd.DataFrame.from_dict(val_set.labels, orient='index')

In [36]:
label_df[:100]

,steer,throttle,brake,hand_brake,reverse
episode_0099/CameraRGB/000540,-0.054236,0.5,0.0,False,False
episode_0099/CameraRGB/000868,-0.008359,0.0,1.0,False,False
episode_0099/CameraRGB/000770,0.008301,0.0,1.0,False,False
episode_0099/CameraRGB/000037,0.198570,0.5,0.0,False,False
episode_0032/CameraRGB/000563,0.075024,0.0,1.0,False,False
...,...,...,...,...,...
episode_0099/CameraRGB/000288,0.010730,0.0,1.0,False,False
episode_0002/CameraRGB/000294,0.108044,0.0,1.0,False,False
episode_0010/CameraRGB/000338,-0.032366,0.0,1.0,False,False
episode_0010/CameraRGB/000663,0.066639,0.5,0.0,False,False


In [25]:
label_df['throttle'].value_counts()

0.0    8084
0.5    8070
1.0    3846
Name: throttle, dtype: int64

In [66]:
net = PilotNet()
net.load_state_dict(torch.load('../checkpoints/net_epoch_0.pt'))

<All keys matched successfully>

# epoch 20 train

In [65]:
with torch.no_grad():
    for x, y in train_loader:
        output = net(x)

        r = pd.DataFrame({'y_true': y.squeeze().tolist(), 'y_pred': output.squeeze().tolist()})
        r['diff'] = abs(r['y_true'] - r['y_pred'])
        print(r)
        break

      y_true        y_pred      diff
0   0.058658  2.581488e-02  0.032843
1  -0.013193  8.729206e-03  0.021922
2  -0.505007 -5.568654e-01  0.051858
3   0.064320  1.915296e-02  0.045167
4   0.406180  5.867173e-01  0.180538
5   0.076585  8.532800e-02  0.008743
6   0.049195  9.027996e-03  0.040167
7   0.003282  5.079129e-02  0.047510
8   0.096242  1.599441e-02  0.080248
9   0.007911  8.139758e-03  0.000229
10 -0.098238 -2.857780e-02  0.069660
11 -0.091972 -5.719018e-03  0.086253
12  0.082692  3.501773e-07  0.082692
13  0.050818 -3.543496e-04  0.051172
14  0.044387  7.112212e-03  0.037274
15 -0.007069  6.467004e-02  0.071739
16  0.170292  1.519162e-01  0.018376
17 -0.069468  9.317694e-03  0.078785
18 -0.048684  5.133658e-03  0.053818
19 -0.039599  3.339306e-02  0.072992
20 -0.096481 -5.023616e-02  0.046245
21  0.009432  1.059375e-02  0.001162
22 -0.040005 -1.420758e-02  0.025797
23 -0.022517  3.927867e-02  0.061795
24  0.035475 -1.332227e-02  0.048798
25  0.046424  4.065935e-02  0.005764
2

# epoch 20 val

In [64]:
with torch.no_grad():
    for x, y in val_loader:
        output = net(x)

        r = pd.DataFrame({'y_true': y.squeeze().tolist(), 'y_pred': output.squeeze().tolist()})
        r['diff'] = abs(r['y_true'] - r['y_pred'])
        print(r)
        break

      y_true    y_pred      diff
0  -0.096465  0.002446  0.098911
1   0.036719 -0.007849  0.044568
2   0.012999  0.003305  0.009695
3  -0.056147  0.025382  0.081529
4   0.015840  0.003238  0.012602
5   0.037365 -0.001571  0.038936
6  -0.027121  0.003138  0.030260
7  -0.098222  0.003155  0.101378
8   0.662616  0.038257  0.624359
9  -0.031648 -0.007036  0.024612
10  0.079143 -0.000120  0.079264
11 -0.095463  0.016002  0.111465
12 -0.051396 -0.006206  0.045190
13  0.060515 -0.007523  0.068038
14  0.079789 -0.006781  0.086571
15  0.092479  0.256682  0.164203
16  0.059257  0.003253  0.056004
17 -0.003085  0.003133  0.006218
18  0.087389  0.001122  0.086267
19 -0.009801  0.000913  0.010714
20 -0.027202  0.003004  0.030206
21 -0.029597  0.002930  0.032527
22  0.013737  0.020257  0.006519
23  0.064578  0.004163  0.060414
24  0.024955  0.063843  0.038888
25 -0.048009 -0.006515  0.041494
26  0.045181  0.021176  0.024006
27  0.021276 -0.002485  0.023761
28  0.068274  0.003191  0.065083
29  0.1070

# bad model train

In [67]:
with torch.no_grad():
    for x, y in train_loader:
        output = net(x)

        r = pd.DataFrame({'y_true': y.squeeze().tolist(), 'y_pred': output.squeeze().tolist()})
        r['diff'] = abs(r['y_true'] - r['y_pred'])
        print(r)
        break

      y_true    y_pred      diff
0  -0.013922 -0.023099  0.009177
1  -0.020676 -0.002279  0.018397
2  -0.110419 -0.031066  0.079353
3  -0.178152 -0.011888  0.166264
4   0.091041 -0.012487  0.103528
5  -0.048897 -0.004273  0.044624
6   0.215275  0.000623  0.214652
7   0.051685 -0.015844  0.067529
8   0.342252 -0.020937  0.363189
9   0.052935  0.018421  0.034514
10 -0.020017 -0.012733  0.007284
11 -0.055528  0.003400  0.058928
12 -0.099684 -0.009167  0.090517
13  0.018317 -0.007065  0.025382
14 -0.053122 -0.020352  0.032770
15  0.192025 -0.017073  0.209098
16 -0.537225 -0.048969  0.488255
17  0.040313 -0.006904  0.047217
18  0.037958 -0.003784  0.041742
19 -0.080555 -0.011440  0.069115
20 -0.087342 -0.006249  0.081093
21  0.099522 -0.001499  0.101022
22  0.124678 -0.008667  0.133345
23 -0.126579 -0.016356  0.110223
24 -0.040045 -0.003685  0.036359
25  0.048138 -0.015305  0.063443
26  0.121413 -0.022048  0.143461
27  0.000154 -0.004658  0.004812
28  0.016758 -0.024825  0.041583
29 -0.0382

# bad model val

In [68]:
with torch.no_grad():
    for x, y in val_loader:
        output = net(x)

        r = pd.DataFrame({'y_true': y.squeeze().tolist(), 'y_pred': output.squeeze().tolist()})
        r['diff'] = abs(r['y_true'] - r['y_pred'])
        print(r)
        break

      y_true    y_pred      diff
0  -0.096465  0.020038  0.116503
1   0.036719  0.010400  0.026319
2   0.012999  0.019825  0.006825
3  -0.056147  0.006019  0.062166
4   0.015840  0.019627  0.003787
5   0.037365 -0.002726  0.040091
6  -0.027121  0.020044  0.047165
7  -0.098222  0.019896  0.118119
8   0.662616 -0.008854  0.671470
9  -0.031648 -0.022534  0.009114
10  0.079143  0.001788  0.077355
11 -0.095463  0.033830  0.129293
12 -0.051396 -0.010124  0.041271
13  0.060515 -0.022632  0.083147
14  0.079789 -0.022563  0.102352
15  0.092479  0.012458  0.080022
16  0.059257  0.019767  0.039490
17 -0.003085  0.019919  0.023004
18  0.087389  0.016391  0.070997
19 -0.009801  0.018028  0.027828
20 -0.027202  0.019875  0.047077
21 -0.029597  0.019636  0.049234
22  0.013737 -0.021461  0.035198
23  0.064578  0.020790  0.043788
24  0.024955 -0.000538  0.025492
25 -0.048009 -0.022761  0.025247
26  0.045181  0.021366  0.023815
27  0.021276  0.006546  0.014730
28  0.068274  0.020157  0.048117
29  0.1070